<a href="https://colab.research.google.com/github/Maryamshahzadi-123/Quarter-2/blob/main/Project_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": [],
      "authorship_tag": "ABX9TyMw5vqJsIsVBgmUbUSpjs+U",
      "include_colab_link": true
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "view-in-github",
        "colab_type": "text"
      },
      "source": [
        "<a href=\"https://colab.research.google.com/github/eshafayyazz/Quarter02/blob/main/Project_02_LangChain_RAG.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 1,
      "metadata": {
        "id": "JF071xfAYYsL",
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "outputId": "47fd0760-4808-44fe-d438-dff19e78f5ab"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\u001b[?25l   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m0.0/41.7 kB\u001b[0m \u001b[31m?\u001b[0m eta \u001b[36m-:--:--\u001b[0m\r\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m41.7/41.7 kB\u001b[0m \u001b[31m2.3 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",
            "\u001b[?25h\u001b[?25l   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m0.0/1.3 MB\u001b[0m \u001b[31m?\u001b[0m eta \u001b[36m-:--:--\u001b[0m\r\u001b[2K   \u001b[91m━━━━━━━━━━━━━━━━━━━━━\u001b[0m\u001b[90m╺\u001b[0m\u001b[90m━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m0.7/1.3 MB\u001b[0m \u001b[31m20.4 MB/s\u001b[0m eta \u001b[36m0:00:01\u001b[0m\r\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m1.3/1.3 MB\u001b[0m \u001b[31m19.4 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",
            "\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m427.3/427.3 kB\u001b[0m \u001b[31m21.3 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",
            "\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m87.5/87.5 kB\u001b[0m \u001b[31m5.8 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",
            "\u001b[2K   \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m50.3/50.3 kB\u001b[0m \u001b[31m3.3 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",
            "\u001b[?25h"
          ]
        }
      ],
      "source": [
        "!pip install -qU langchain-pinecone langchain_google_genai"
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "from google.colab import userdata\n",
        "from pinecone import Pinecone, ServerlessSpec\n",
        "Pinecone_Api_key = userdata.get('PINECONE_API_KEY')\n",
        "pc = Pinecone(api_key=Pinecone_Api_key )"
      ],
      "metadata": {
        "id": "VFYVvXUbg2Og"
      },
      "execution_count": 2,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "index_name = \"langchain-rag-project\"\n",
        "pc.create_index(\n",
        "        name=index_name,\n",
        "        dimension=3072,\n",
        "        metric=\"cosine\",\n",
        "        spec=ServerlessSpec(cloud=\"aws\", region=\"us-east-1\"),\n",
        "    )\n",
        "index = pc.Index(index_name)"
      ],
      "metadata": {
        "id": "guWrd6chh78T"
      },
      "execution_count": 5,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "import getpass\n",
        "import os\n",
        "from google.colab import userdata\n",
        "GEMINI_KEY=userdata.get('GOOGLE_API_KEY')\n",
        "os.environ[\"GOOGLE_API_KEY\"]=GEMINI_KEY"
      ],
      "metadata": {
        "id": "j0SsHaOhieUB"
      },
      "execution_count": 6,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "from langchain_google_genai import GoogleGenerativeAIEmbeddings\n",
        "# The model name should be \"models/embedding-001\"\n",
        "# instead of \"models\\embedding-001\".\n",
        "# The forward slash is the correct delimiter for model names.\n",
        "embeddings = GoogleGenerativeAIEmbeddings(model=\"models/embedding-001\")"
      ],
      "metadata": {
        "id": "rOf_oK5XjH0O"
      },
      "execution_count": 15,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "from langchain_pinecone import PineconeVectorStore\n",
        "vector_store = PineconeVectorStore(index=index, embedding=embeddings)"
      ],
      "metadata": {
        "id": "t26IwZyukt7u"
      },
      "execution_count": 16,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "pc.delete_index(index_name)  # Delete the existing index\n",
        "\n",
        "pc.create_index(\n",
        "        name=index_name,\n",
        "        dimension=768,  # Set the dimension to 768 to match the embeddings\n",
        "        metric=\"cosine\",\n",
        "        spec=ServerlessSpec(cloud=\"aws\", region=\"us-east-1\"),\n",
        "    )\n",
        "index = pc.Index(index_name)"
      ],
      "metadata": {
        "id": "w-3577JRsm_k"
      },
      "execution_count": 19,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "from uuid import uuid4\n",
        "\n",
        "from langchain_core.documents import Document\n",
        "\n",
        "document_1 = Document(\n",
        "    page_content=\"I had chocalate chip pancakes and scrambled eggs for breakfast this morning.\",\n",
        "    metadata={\"source\": \"tweet\"},\n",
        ")\n",
        "\n",
        "document_2 = Document(\n",
        "    page_content=\"The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.\",\n",
        "    metadata={\"source\": \"news\"},\n",
        ")\n",
        "\n",
        "document_3 = Document(\n",
        "    page_content=\"Building an exciting new project with LangChain - come check it out!\",\n",
        "    metadata={\"source\": \"tweet\"},\n",
        ")\n",
        "\n",
        "document_4 = Document(\n",
        "    page_content=\"Robbers broke into the city bank and stole $1 million in cash.\",\n",
        "    metadata={\"source\": \"news\"},\n",
        ")\n",
        "\n",
        "document_5 = Document(\n",
        "    page_content=\"Wow! That was an amazing movie. I can't wait to see it again.\",\n",
        "    metadata={\"source\": \"tweet\"},\n",
        ")\n",
        "\n",
        "document_6 = Document(\n",
        "    page_content=\"Is the new iPhone worth the price? Read this review to find out.\",\n",
        "    metadata={\"source\": \"website\"},\n",
        ")\n",
        "\n",
        "document_7 = Document(\n",
        "    page_content=\"The top 10 soccer players in the world right now.\",\n",
        "    metadata={\"source\": \"website\"},\n",
        ")\n",
        "\n",
        "document_8 = Document(\n",
        "    page_content=\"LangGraph is the best framework for building stateful, agentic applications!\",\n",
        "    metadata={\"source\": \"tweet\"},\n",
        ")\n",
        "\n",
        "document_9 = Document(\n",
        "    page_content=\"The stock market is down 500 points today due to fears of a recession.\",\n",
        "    metadata={\"source\": \"news\"},\n",
        ")\n",
        "\n",
        "document_10 = Document(\n",
        "    page_content=\"I have a bad feeling I am going to get deleted :(\",\n",
        "    metadata={\"source\": \"tweet\"},\n",
        ")\n",
        "\n",
        "documents = [\n",
        "    document_1,\n",
        "    document_2,\n",
        "    document_3,\n",
        "    document_4,\n",
        "    document_5,\n",
        "    document_6,\n",
        "    document_7,\n",
        "    document_8,\n",
        "    document_9,\n",
        "    document_10,\n",
        "]\n",
        "uuids = [str(uuid4()) for _ in range(len(documents))]\n",
        "vector_store.add_documents(documents=documents, ids=uuids)"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "_dYBjlKQqhlk",
        "outputId": "ce4229e7-96e1-41f2-a1de-b765db9b015e"
      },
      "execution_count": 20,
      "outputs": [
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "['ab267fbf-b039-4aeb-bc7a-84cac25067c7',\n",
              " 'fb7095f6-a5d1-4f73-ab4d-8f41742e4c52',\n",
              " '14eade93-4f8e-477f-85d7-d2125f622de2',\n",
              " '2ee98688-030e-4076-8d71-4064ea0f66e1',\n",
              " '9b819c5d-4710-449b-8b18-f243acadff2b',\n",
              " 'ee0ef3ef-db6d-4c27-b70e-aa47933783a4',\n",
              " 'a50cf6da-5f44-4ead-a8e4-11d81a7d0890',\n",
              " '189c92c8-4352-4d43-bdfb-12a8bcb0bb8f',\n",
              " '29d28501-d372-421d-8e04-79e46ee9b126',\n",
              " 'b60aeac1-92e0-419e-888e-0ee13b49ae36']"
            ]
          },
          "metadata": {},
          "execution_count": 20
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "from langchain_google_genai import ChatGoogleGenerativeAI\n",
        "from langchain.prompts import PromptTemplate"
      ],
      "metadata": {
        "id": "cq0iZJ-CthW7"
      },
      "execution_count": 21,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "def user_answer(question: str):\n",
        "  results = vector_store.similarity_search(question, k=7) # This was defined but never used\n",
        "  llm = ChatGoogleGenerativeAI(\n",
        "      model=\"gemini-2.0-flash-exp\",\n",
        "      api_key=GEMINI_KEY,\n",
        "      temperature=0.7, #Typo in the code, temprarture should be temperature\n",
        "      max_tokens=100,\n",
        "  )\n",
        "  prompt1 = PromptTemplate(\n",
        "      input_variables=[\"question\", \"vector_results\"], # Add vector_results as an input variable\n",
        "      template=\"Using this data {vector_results}, answer the following question: {question}\",\n",
        "  )\n",
        "  chain1 = prompt1 | llm\n",
        "  # Pass results to the invoke call as vector_results\n",
        "  final_answer = chain1.invoke({\"question\": question, \"vector_results\": results})\n",
        "  return final_answer"
      ],
      "metadata": {
        "id": "GREhs0wHtxyy"
      },
      "execution_count": 29,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "response=user_answer(\"What is the weather forecast for tomorrow?\")\n",
        "print(\"Answer:\",response.content)"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "C3h5GcVFxo-t",
        "outputId": "8c326b43-c275-4842-a83a-9f28132b0432"
      },
      "execution_count": 30,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Answer: The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.\n"
          ]
        }
      ]
    }
  ]
}

NameError: name 'true' is not defined